In [54]:
import pandas as pd
import torch

df = pd.read_csv("../data/observations-497968.csv")

In [55]:
model = torch.hub.load("pytorch/vision:v0.10.0", "mobilenet_v2", pretrained=True)

Using cache found in /Users/kaiose/.cache/torch/hub/pytorch_vision_v0.10.0


In [56]:
labels = {"Plantae": 0, "Insecta": 1, "Aves": 2, "Mammalia": 3}
label_weights = df["iconic_taxon_name"].value_counts().values.tolist()
class_weights = torch.tensor(label_weights, dtype=torch.float)
df["taxon_label"] = df["iconic_taxon_name"].map(labels)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch import nn, optim
from PIL import Image
import requests


class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        label = self.dataframe.iloc[idx, 1]
        img_url = self.dataframe.iloc[idx, 2]
        image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)


transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # MobileNet input size is 224x224
        transforms.ToTensor(),  # Convert image to a PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ]
)

df = df[["id", "taxon_label", "image_url"]]

dataset = CustomImageDataset(dataframe=df, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

model = models.mobilenet_v2(pretrained=True)

num_classes = df["taxon_label"].nunique()  # Get the number of unique classes

# Modify the final fully connected layer (classifier) of MobileNetV2
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Finished Fine-tuning the model")
torch.save(model.state_dict(), "../data/mobilenetft.pth")